In [103]:
`wget http://www.d.umn.edu/~tpederse/Data/Sval2.keys.tar.gz` |> run

--2016-07-09 00:25:26--  http://www.d.umn.edu/~tpederse/Data/Sval2.keys.tar.gz
Resolving www.d.umn.edu (www.d.umn.edu)... 131.212.109.109, 2607:ea00:200:109::10
Connecting to www.d.umn.edu (www.d.umn.edu)|131.212.109.109|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42692 (42K) [application/x-gzip]
Saving to: 'Sval2.keys.tar.gz’

     0K .......... .......... .......... .......... .         100% 34.5K=1.2s

2016-07-09 00:25:28 (34.5 KB/s) - 'Sval2.keys.tar.gz’ saved [42692/42692]



In [1]:
using WordNet
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using Iterators

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31


In [2]:
db = DB("data/corpora/WordNet-2.1/")

WordNet.DB

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [39]:
WordNet.SYNSET_TYPES

Dict{Char,AbstractString} with 4 entries:
  'n' => "noun"
  'a' => "adj"
  'r' => "adv"
  'v' => "verb"

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


1-element Array{WordNet.Synset,1}:
 (s) vi, half dozen, half-dozen, six, 6 (denoting a quantity consisting of six items or units)

In [45]:
lemma.word

"six"

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [31]:
methods(maximum)

# 8 methods for generic function "maximum":
maximum(r::UnitRange) at range.jl:371
maximum(r::Range) at range.jl:373
maximum(f::Function, A::AbstractArray, region) at reducedim.jl:252
maximum(f::Union{DataType,Function}, a) at reduce.jl:283
maximum(B::BitArray) at bitarray.jl:1691
maximum{T<:Real}(x::AbstractSparseArray{T,Ti<:Any,1}) at sparse/sparsevector.jl:1141
maximum(A::AbstractArray, region) at reducedim.jl:254
maximum(a) at reduce.jl:286

In [4]:
ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.model");

In [5]:
function normal_probs(logprobs::Vector)
    ret = copy(logprobs)
    max_lp = maximum(logprobs)
    ret.-=max_lp #Bring closer to zero
    map!(exp,ret)
    denom = sum(ret)
    ret./=denom
    ret
end
function weighted_average(logprobs, embeddings)
    ret = zeros(first(embeddings))
    for (weight, embedding) in zip(normal_probs(logprobs), embeddings)
        ret.+= weight.*embedding
    end
    ret
end

function all_logprobs_of_context(embed, context, middle_word)
    [Query.logprob_of_context(embed, context,wv; skip_oov=true, normalise_over_length=true)
        for wv in embed.embedding[middle_word]]
end

function synthesize_embedding(ee,context, middle_word)
    logprobs = all_logprobs_of_context(ee, context,middle_word)
    weighted_average(logprobs, ee.embedding[middle_word])
end

synthesize_embedding (generic function with 1 method)

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [6]:
function poormans_tokenize(source)
    cleaned = filter(s->!ispunct(s), lowercase(source))
    map(string,  split(cleaned))::Vector{SubString{String}}
end

function punctuation_space_tokenize(source)
    preprocced = lowercase(source)
    pass1=replace(preprocced,r"[[:punct:]]*[[:space:]][[:punct:]]*"," ")
    pass2=replace(pass1,r"[[:punct:]]*$|^[[:punct:]]*","")
    split(pass2)
end



punctuation_space_tokenize (generic function with 1 method)

In [7]:
function lexical_informed_embeddings(db,ee, word)
    source_wvs = ee.embedding[word]
    target_synsets = Base.flatten((haskey(db.lemmas[pos], word) ? synsets(db, db[pos, word]) : WordNet.Synset[] 
        for pos in keys(WordNet.SYNSET_TYPES))) 

    if length(source_wvs) == 1
        #Use MCWS
        lemma = db['n', lemma_word]
        target_synsets = synsets(db, lemma);
        score, index = findmax((get(ss.word_counts,lemma_word,0)+0.1sum(values(ss.word_counts))) for ss in target_synsets)
        target_synsets=[target_synsets[index]] # Throw out others
    else
            
    synsets4vectors = Dict{eltype(source_wvs), Synset}()
    for synset::Synset in target_synsets
            context::Vector{SubString{String}} = collect(filter!(w->w!=word, punctuation_space_tokenize(synset.gloss)))
            wv = synthesize_embedding(ee,context, word)
            synsets4vectors[wv] = synset
    end
    synsets4vectors
end

lexical_informed_embeddings (generic function with 1 method)

In [46]:
@time lexical_informed_embeddings(db,ee, "federal")

  0.024479 seconds (102.73 k allocations: 1.726 MB)


Dict{Array{Float32,1},WordNet.Synset} with 6 entries:
  Float32[-0.0537537,0.0682679,-0.0259331,0.157535,-0.272935,0.186007,-0.0694081,-0.0785202,0.199033,-0.126381,-0.0847547,0.128801,-0.183493,-0.0422632,0.0892356,-0.22934,-0.0623469,-0.00149297,0.056994,0.0964768,0.0161398,0.0781163,0.211676,-0.0763543,0.144749,0.216146,0.0675393,0.088937,-0.118281,-0.356966,0.0163716,-0.0818849,0.0128478,-0.0469062,-0.0810498,0.203175,0.0173182,0.0536753,0.0529723,-0.0489318,0.0774052,-0.0222965,-0.0963653,0.0107006,0.0746002,-0.082665,-0.199656,-0.186337,-0.0727619,-0.0921054,0.103729,0.0355965,0.0991122,0.114297,0.0437884,0.0641925,-0.155373,-0.191694,0.106501,0.0299754,0.202594,0.0104121,0.00473919,-0.049686,0.203001,0.149391,-0.0946662,-0.00488328,0.0074259,-0.189963,0.206126,0.0392408,-0.0732178,-0.0386577,0.0215429,0.181814,0.00604329,0.0138382,-0.0173619,-0.0353349,-0.0625751,-0.151501,0.00638058,0.139065,0.130912,0.0440749,0.156332,0.108061,0.0793152,0.0144482,0.140982,0.0163569,-0.061246,0

In [ ]:
#See http://webcache.googleusercontent.com/search?q=cache:b5Ixl2Ds2zcJ:icl.pku.edu.cn/member/yujs/papers/pdf/corpus-format.pdf+&cd=12&hl=en&ct=clnk&gl=au&client=firefox-b-ab


function get_sense_key(synset, lemma):
    index = 0
    try: #try and do better
        index = synset.lemma_names().index(lemma)
    except ValueError:
        pass #couldn't do better
    return synset.lemmas()[index].key()

def eval_wsd(wsd, text_id, tagged_phrases, indexed_instances):
    lemmas_and_synsets = wsd(tagged_phrases)
    for ii, id in indexed_instances.items():
        #print(id)
        lemma, synset = lemmas_and_synsets[ii]
        comment = "!! lemma="+''.join([i if ord(i) < 128 else ' ' for i in lemma])
        if synset: #If it was even defined in wordnet
            sensekey = get_sense_key(synset,lemma)
            yield text_id, id, sensekey, comment
        else:
            yield text_id, id, comment
        

In [100]:
lemma = db['a', "6"]
ss = synsets(db, lemma)
s1=ss[1]


(s) vi, half dozen, half-dozen, six, 6 (denoting a quantity consisting of six items or units)

In [101]:

fieldvalue(v,fn::Symbol) = eval(Expr(:(.), v, QuoteNode(fn)))
function print_structure(v)
    for fn in fieldnames(v)
        println(fn,"\t", fieldvalue(v,fn))
    end   
end

print_structure(s1)
println("-"^10)
print_structure(lemma)

offset	2263219
lex_filenum	0
word_counts	Dict{AbstractString,Int64}(Pair{AbstractString,Int64}("vi",0),Pair{AbstractString,Int64}("half_dozen",0),Pair{AbstractString,Int64}("half-dozen",0),Pair{AbstractString,Int64}("six",0),Pair{AbstractString,Int64}("6",0))
synset_type	s
pos	a
pointers	WordNet.Pointer[WordNet.Pointer("&",0x00227c1d,'a',"00","00")]
gloss	denoting a quantity consisting of six items or units
----------
word	6
pos	a
tagsense_count	1
synset_offsets	[2263219]
id	172
pointer_syms	AbstractString["&"]


In [102]:
?getfield

search: getfield



```
getfield(value, name::Symbol)
```

Extract a named field from a `value` of composite type. The syntax `a.b` calls `getfield(a, :b)`.


In [ ]:
using LightXML

In [ ]:
immutable wsd_challenge
    id
    word
    lemma
    pos
    context
end

function load_challenges(xml_file="data/corpora/wsd/semeval2007_t7/test/eng-coarse-all-words.xml")
    xdoc = parse_file(xml_file)
    xroot = root(xdoc)
    Task() do
        for text_node in child_elements(xroot)
            #@show text_node
            #text = child_nodes(text_node) |> collect
            #println(text)
            for sentence_node in child_elements(text_node)
                sentence = punctuation_space_tokenize(content(sentence_node))

                for lemma_node in child_elements(sentence_node)
                    word = content(lemma_node) |> lowercase
                    lemma = attribute(lemma_node,"lemma")|> lowercase
                    pos = attribute(lemma_node,"pos")
                    id = attribute(lemma_node,"id")
                    context = filter(w->w!=word, sentence)
                    produce(wsd_challenge(id, word, lemma, pos, context))
                end
            end
        end
    end
end
challenges = load_challenges() |> collect